### 2-1. Nate 뉴스기사 제목 스크래핑하기

In [3]:
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from IPython.display import Image, display

# 뉴스 섹션과 해당 mid 값을 매핑하는 딕셔너리
section_dict = {
    '최신뉴스': 'n0100',
    '정치': 'n0200',
    '경제': 'n0300',
    '사회': 'n0400',
    '세계': 'n0500',
    'IT/과학': 'n0600'
}

def get_nate_news_data(section_name):
    
    # 섹션 이름의 유효성 검사
    section_code = section_dict.get(section_name)
    if not section_code:
        print(f"'{section_name}' 섹션은 존재하지 않거나 지원되지 않습니다. "
              "다음 중 하나를 선택하세요: " + ", ".join(section_dict.keys()))
        return

    #  요청할 뉴스 섹션의 URL 생성
    base_url = "https://news.nate.com"
    news_section_url = f'{base_url}/recent?mid={section_code}'

    print(f'\n--- {section_name} 뉴스 스크래핑 시작 ({news_section_url}) ---')

    request_headers = {
        'Referer': news_section_url, # 요청 출처를 명시하여 정상적인 접근처럼 보이게 함
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Whale/4.32.315.22 Safari/537.36"
    }

    try:
        # 웹 페이지 요청 및 인코딩 설정
        response = requests.get(news_section_url, headers=request_headers)
        response.encoding = 'EUC-KR' # 네이트 뉴스 인코딩에 맞춰 설정
        response.raise_for_status() # HTTP 오류(4xx, 5xx) 발생 시 예외 발생

        soup = BeautifulSoup(response.text, 'html.parser')

        # 뉴스 항목 선택 
        news_entries = soup.select('div.postSubjectContent div.mlt01')

        if not news_entries:
            print(f"'{section_name}' 섹션에서 뉴스 항목을 찾을 수 없습니다. 웹사이트 구조 변경 여부를 확인하세요.")
            return

        #  각 뉴스 항목에서 정보 추출 및 출력
        for index, entry in enumerate(news_entries):
            print(f"\n--- 기사 #{index + 1} ---")

            # 기사 제목 추출
            title_tag = entry.select_one('h2.tit')
            article_title = title_tag.text.strip() if title_tag else "제목 없음"
            print(f"기사 제목: {article_title}")

            # 기사 링크 추출
            link_tag = entry.find('a') 
            # urljoin을 사용하여 상대 경로를 절대 경로로 변환 
            article_link = urljoin(base_url, link_tag['href']) if link_tag and 'href' in link_tag.attrs else "링크 없음"
            print(f"기사 링크: {article_link}")

            # 이미지 정보 추출 및 출력
            image_tag = link_tag.select_one('img[src]') if link_tag else None 

            # 이미지 태그가 존재하고 'src' 속성이 있는지 확인
            if image_tag and 'src' in image_tag.attrs:
                image_src = image_tag['src']
                
                # 이미지 src가 //로 시작하는 프로토콜 상대 경로일 경우 'https:' 추가
                if image_src.startswith('//'):
                    image_src = 'https:' + image_src
                
                # urljoin을 사용하여 이미지의 전체 URL 생성 
                full_image_url = urljoin(base_url, image_src)
                print(f"이미지 URL: {full_image_url}")
                
                # IPython.display를 사용하여 이미지 표시
                try:
                    display(Image(url=full_image_url, width=150)) # 이미지 너비 지정
                except Exception as e:
                    print(f"이미지를 표시하는 데 실패했습니다 ({full_image_url}): {e}")
            else:
                print("이미지: 없음")

    except requests.exceptions.RequestException as e:
        print(f"네트워크 요청 중 오류 발생: {e}")
    except Exception as e:
        print(f"데이터 처리 중 예상치 못한 오류 발생: {e}")

    print(f'--- {section_name} 뉴스 스크래핑 완료 ---')



In [4]:
#get_nate_news_data('section') 원하는기사 출력
get_nate_news_data('경제')


--- 경제 뉴스 스크래핑 시작 (https://news.nate.com/recent?mid=n0300) ---

--- 기사 #1 ---
기사 제목: 프로야구, 역대 최소경기 800만 관중 돌파…최다 매진 타이기록
기사 링크: https://news.nate.com/view/20250725n00571?mid=n0300
이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/jt/2025/07/25/20250725011513563001.png



--- 기사 #2 ---
기사 제목: 공항서 되돌아온 부총리
기사 링크: https://news.nate.com/view/20250725n00409?mid=n0300
이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/jo/2025/07/25/c2da8257-90a8-4a4c-a6f4-917f40bdc86b.jpg



--- 기사 #3 ---
기사 제목: "내년까지 힘들 수도" 머스크 CEO 시인에 테슬라 주가 장중 8%대 폭락
기사 링크: https://news.nate.com/view/20250725n00545?mid=n0300
이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/sg/2025/07/25/20250725500451.jpg



--- 기사 #4 ---
기사 제목: EU "美와 관세협상 체결 '가시권'"
기사 링크: https://news.nate.com/view/20250725n00539?mid=n0300
이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/cz/2025/07/25/cz_1753373104807_806323_0.jpg



--- 기사 #5 ---
기사 제목: 구윤철 출국전 공항서 발길 돌려…트럼프 '韓 카드' 불만 가능성
기사 링크: https://news.nate.com/view/20250725n00001?mid=n0300
이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/no/2025/07/25/SSC_20250724175415_V.jpg


...


--- 기사 #16 ---
기사 제목: '속도전' 배경훈 장관 "AI 생태계 구축, 2~3년 내 승부 봐야"
기사 링크: https://news.nate.com/view/20250725n00401?mid=n0300
이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/kx/2025/07/25/2025072420080320635_1753355283_1753339396.jpg



--- 기사 #17 ---
기사 제목: LS일렉트릭, '국가전력망 핵심' HVDC 국산화 속도
기사 링크: https://news.nate.com/view/20250725n00395?mid=n0300
이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/kx/2025/07/25/2025072420070320630_1753355223_1753342129.jpg



--- 기사 #18 ---
기사 제목: 경제8단체 "상법 추가 개정, 경영권 위협 무방비 노출"
기사 링크: https://news.nate.com/view/20250725n00392?mid=n0300
이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/kx/2025/07/25/2025072420060420627_1753355164_1753345068.jpg



--- 기사 #19 ---
기사 제목: "이 제품 관세 붙나요?"…美 관세 유예기간 임박에 문의 급증
기사 링크: https://news.nate.com/view/20250725n00387?mid=n0300
이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/kx/2025/07/25/2025072421112520823_1753359085_1753346867.jpg



--- 기사 #20 ---
기사 제목: 미 관세 직격탄 맞은 현대차…2분기 영업익 15.8% 줄었다
기사 링크: https://news.nate.com/view/20250725n00386?mid=n0300
이미지: 없음
--- 경제 뉴스 스크래핑 완료 ---


### 2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기

In [5]:
import requests
import os
from bs4 import BeautifulSoup

def download_one_episode(title, no, url): 
   
    print(f"'{title}' 웹툰의 {no}회차({url}) 이미지 다운로드를 시작합니다...")

    # 이미지를 저장할 디렉토리 경로를 설정합니다.
    # 'img/웹툰제목/회차번호' 형태로 생성됩니다.
    image_directory = os.path.join('img', title, str(no))

    # 이미지 저장 폴더가 없으면 새로 생성하고, 이미 존재하면 아무 작업도 하지 않습니다.
    os.makedirs(image_directory, exist_ok=True)
    print(f"이미지 저장 경로: '{image_directory}'")

    
    request_headers = {
        'referer': url,
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Whale/4.32.315.22 Safari/537.36"
    
                       }

    try:
        page_response = requests.get(url)
        page_response.encoding = 'EUC-KR'

        # HTTP 응답 코드가 200 (성공)이 아니면 오류 메시지를 출력하고 함수를 종료합니다.
        if not page_response.ok:
            print(f"오류: 웹 페이지 접근 실패. 상태 코드: {page_response.status_code}")
            return # 함수 종료
    except requests.exceptions.RequestException as e:
        print(f"오류: 웹 페이지 요청 중 네트워크 문제가 발생했습니다: {e}")
        return # 함수 종료

    # HTML을 파싱하고 이미지 URL을 추출
    soup = BeautifulSoup(page_response.text, 'html.parser')
    
    # 'img' 태그 중 'src' 속성에 'IMAG01'이 포함된 모든 이미지를 선택
    image_urls = [img_tag['src'] for img_tag in soup.select("img[src*='IMAG01']")]

    if not image_urls:
        print(f"경고: '{title}' 웹툰 {no}회차에서 다운로드할 이미지를 찾을 수 없습니다. "
              "페이지 구조가 변경되었거나, 이미지가 없거나, 접근 제한이 있을 수 있습니다.")
        return 

    #  추출된 각 이미지 URL을 통해 이미지를 다운로드하고 저장
    for img_url in image_urls:
        try:
            image_response = requests.get(img_url, headers=request_headers)
            
            if image_response.ok:
                image_data = image_response.content # 이미지 데이터를 바이너리 형태로 가져옴
                
                # 이미지 파일명을 URL에서 추출하고, 저장할 파일의 전체 경로를 생성
                file_name = os.path.basename(img_url)
                file_path = os.path.join(image_directory, file_name)
                
                # 파일을 바이너리 쓰기 모드('wb')로 열어 이미지 데이터를 저장
                with open(file_path, 'wb') as file:
                    file.write(image_data)
                print(f"성공: '{file_path}'에 이미지 저장 완료 ({len(image_data):,} bytes)")
            else:
                print(f"오류: 이미지 다운로드 실패. 상태 코드: {image_response.status_code} ({img_url})")
        except requests.exceptions.RequestException as e:
            print(f"오류: 이미지 ({img_url}) 다운로드 중 네트워크 문제가 발생했습니다: {e}")
        except IOError as e:
            print(f"오류: 이미지 파일 '{file_path}' 저장 중 문제가 발생했습니다: {e}")

    print(f"'{title}' 웹툰 {no}회차의 모든 이미지 다운로드가 완료되었습니다.")


download_one_episode('일렉시드', 341, 'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed')

'일렉시드' 웹툰의 341회차(https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed) 이미지 다운로드를 시작합니다...
이미지 저장 경로: 'img\일렉시드\341'
성공: 'img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_1.jpg'에 이미지 저장 완료 (87,143 bytes)
성공: 'img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_2.jpg'에 이미지 저장 완료 (256,127 bytes)
성공: 'img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_3.jpg'에 이미지 저장 완료 (184,536 bytes)
성공: 'img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_4.jpg'에 이미지 저장 완료 (182,867 bytes)
성공: 'img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_5.jpg'에 이미지 저장 완료 (112,615 bytes)
성공: 'img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_6.jpg'에 이미지 저장 완료 (169,889 bytes)
성공: 'img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_7.jpg'에 이미지 저장 완료 (157,876 bytes)
성공: 'img\일렉시드\341\20250311184953_812848a288eb3e6ce6efe904abd0ef68_IMAG01_8.jpg'에 이미지 저장 완료 (181,837 bytes)
성공: 'img\일렉시